# Creating a Basic Blockchain with Python

In [33]:
import hashlib
from datetime import datetime

## Part 1: A Simple Blockchain

#### Remember:
- The blockchain is meant to add new blocks
- It is not meant to change existing blocks
- It is not meant to delete existing blocks

In [34]:
class Block:
    def __init__(self, timestamp, data, previous_hash = ''):
        # tells us when the block was created
        self.timestamp = timestamp
        # data can be anything .. for crypto currency we might store
        # details of a transaction such as how much money was transferred 
        # who the sender is and who the receiver is!
        self.data = str(data)
        # the hash of the previous block
        # imporant for the integrity of the blockchain
        self.previous_hash = previous_hash
        # the hash of the current block
        self.blockhash = self.calculate_hash()
        
    def calculate_hash(self):
        # calculates the hash of the current block
        # takes the properties of this block and creates a hash for them!
        concat = self.previous_hash + self.timestamp + self.data 
        return hashlib.sha224(concat.encode('utf-8')).hexdigest()
    
    def get_block_data(self):
        # returns the properties of current block
        return {
            'timestamp': self.timestamp,
            'data': self.data,
            'previous_hash': self.previous_hash,
            'hash': self.blockhash
        }
   

class BlockChain:
    def __init__(self):
        # the blockchain is going to be a linked list!
        self.chain = []
        self.chain.append(self.create_genesis_block())

    def create_genesis_block(self):
        # creates the very first block in the blockchain
        return Block(get_timestamp(), ["Genesis Block"], "0")
    
    def get_latest_block(self):
        # returns the very last block in the blockchain
        return self.chain[-1]
    
    def add_block(self, new_block):
        # adds a new block to the blockchain
        # set the previous hash
        new_block.previous_hash = self.get_latest_block().blockhash
        # recalculate the hash of the current block because we've changed its previous_hash
        # every time we change something in the block we need to recalculate its hash
        new_block.blockhash = new_block.calculate_hash()
        # add the block to the blockchain
        self.chain.append(new_block)
        
    def is_chain_valid(self):
        # to verify the integrity of the blockchain
        # we go through all blocks (no need to start from the genesis block)
        for i in range(1, len(self.chain)):
            current = self.chain[i]
            previous = self.chain[i-1]
            # make sure the hash of the current block is still valid
            if current.blockhash != current.calculate_hash():
                return False
            # make sure the current block is connected to the correct previous block
            if current.previous_hash != previous.blockhash:
                return False
        # if we get here then the blockchain is valid
        return True
            

def get_timestamp():
    # A simple function to return the current timestamp!
    date = datetime.now()
    # convert datetime to timestamp
    timestamp = datetime.timestamp(date)
    return str(timestamp)



In [35]:
bc = BlockChain()
d = {
    'Name': 'Noureddin Sadawi',
    "Nationality": "Libyan",
    "Height": 179,
    "Weight": 80
}
bc.add_block(Block(get_timestamp(), d))

d = {
    'Name': 'Josef Baker',
    "Nationality": "British",
    "Height": 185,
    "Weight": 85
}
bc.add_block(Block(get_timestamp(), d))

In [42]:
bc.is_chain_valid()

False

In [40]:
bc.chain[1].blockhash

'7d608de81b9047eefec951353eb8bead2bd3af41dddc060e619d8592'

In [43]:
bc.chain[1].data = "Test"
# even if you recalculate the hash of this block, the blockchain is still invalid
# because the relationship with the previous block is now broken. 
bc.chain[1].blockhash = bc.chain[1].calculate_hash()

In [46]:
bc.is_chain_valid()

False

In [47]:
bc.chain[1].blockhash

'89e69766ae8a1d69d4baaf2f3232f0b9d8735eaf5a7b107e91f8fa4f'

## Part 2: Proof of Work



#### Extension:
- Now we can create a transaction, compute its hash and add it to the chain  
- Computers can do that very quickly but we don't want that!
- We don't want large numbers of blocks added to the blockchain in a short period of time
- Also, currently we can change a block and recalculate the hashes of the blocks after it and the blockchain will still be valid
- We don't want that!
- To avoid that, blockchains have something called Proof of Work!
- With this mechanism you have to prove that you have put in a lot of computing power into making a Block
- This is called Mining
- BitCoin, for example, requires the hash of a block to begin with a certain number of 0's

- Because we can't influence the output of a hash function, we must try a large number of times and hope to generate a hash with the required number of leading 0's
- This requires a lot of computing power
- It is known as the difficulty level
- It helps in controlling the number of created valid blocks in a certain period of time
- Bitcoin aims for a new block every 10 minutes
- As computers get faster over time, they'll require less time to mine a new block
- To compensate for that, the difficulty level is increased



In [48]:
class Block:
    def __init__(self, timestamp, data, previous_hash = ''):
        # tells us when the block was created
        self.timestamp = timestamp
        # data can be anything .. for crypto currency we might store
        # details of a transaction such as how much money was transferred 
        # who the sender is and who the receiver is!
        self.data = str(data)
        # the hash of the previous block
        # imporant for the integrity of the blockchain
        self.previous_hash = previous_hash
        # the nonce is needed for mining blocks
        self.nonce = 0
        # the hash of the current block
        self.blockhash = self.calculate_hash()
        
        
    def calculate_hash(self):
        # calculates the hash of the current block
        # takes the properties of this block and creates a hash for them!
        concat = self.previous_hash + self.timestamp + self.data + str(self.nonce)
        return hashlib.sha224(concat.encode('utf-8')).hexdigest()
    
    def mine_block(self, difficulty):
        # here we see the benefit of the nonce, we increment it until we get the right hash
        # this is because the hash can always be the same with the same block contents
        while self.blockhash[:difficulty] != "0" * difficulty:
            self.nonce = self.nonce + 1
            self.blockhash = self.calculate_hash()
    
    def get_block_data(self):
        return {
            'timestamp': self.timestamp,
            'data': self.data,
            'previous_hash': self.previous_hash,
            'hash': self.blockhash,
            'nonce': self.nonce
        }
   

class BlockChain:
    def __init__(self, difficulty=2):
        # the blockchain is going to be a linked list!
        self.chain = []
        self.chain.append(self.create_genesis_block())
        self.difficulty = difficulty

    def create_genesis_block(self):
        # creates the very first block in the blockchain
        return Block(get_timestamp(), ["Genesis Block"], "0")
    
    def get_latest_block(self):
        # returns the very last block in the blockchain
        return self.chain[-1]
    
    def add_block(self, new_block):
        # adds a new block to the blockchain
        # set the previous hash
        new_block.previous_hash = self.get_latest_block().blockhash
        # now we call the mine_block method
        new_block.mine_block(self.difficulty)
        # once the block is mined it is added to the blockchain
        self.chain.append(new_block)
    
        
    def is_chain_valid(self):
        # to verify the integrity of the blockchain
        # we go through all blocks (no need to start from the genesis block)
        for i in range(1, len(self.chain)):
            current = self.chain[i]
            previous = self.chain[i-1]
            # make sure the hash of the current block is still valid
            if current.blockhash != current.calculate_hash():
                return False
            # make sure the current block is connected to the correct previous block
            if current.previous_hash != previous.blockhash:
                return False
        # if we get here then the blockchain is valid
        return True
            

def get_timestamp():
    # A simple function to return the current timestamp!
    date = datetime.now()
    # convert datetime to timestamp
    timestamp = datetime.timestamp(date)
    return str(timestamp)



In [53]:
bc = BlockChain(difficulty=5)
d = {
    'Name': 'Noureddin Sadawi',
    "Nationality": "Libyan",
    "Height": 179,
    "Weight": 80
}
bc.add_block(Block(get_timestamp(), d))
d = {
    'Name': 'Josef Baker',
    "Nationality": "British",
    "Height": 185,
    "Weight": 85
}
bc.add_block(Block(get_timestamp(), d))

In [54]:
bc.chain[2].get_block_data()

{'timestamp': '1670878054.197167',
 'data': "{'Name': 'Josef Baker', 'Nationality': 'British', 'Height': 185, 'Weight': 85}",
 'previous_hash': '0000072a8bfd4fad7283540c265604cdcff3d536dfe24ce4d20ff287',
 'hash': '000001f3f29875ae2e6d1cc291548586e9af27133c61e2d21d6c4eb6',
 'nonce': 1243832}

In [ ]:
bc.chain[1].blockhash